In [1]:
!pip install pandas requests

In [7]:
import requests
import json
import pandas as pd

# The direct URL to the JSON file on your MinIO console
DATA_URL = 'http://localhost:9001/browser/spotify-sample/mpd.slice.0-999.json'

try:
    # 1. Fetch the data from the URL
    response = requests.get(DATA_URL)
    response.raise_for_status() # Check for HTTP errors (4xx or 5xx)

    # 2. Parse the JSON content
    data = response.json()
    
    print("Data successfully fetched and parsed!")
    
except requests.exceptions.RequestException as e:
    print(f"Error accessing the MinIO server or file: {e}")
    print("Ensure MinIO is running and accessible at http://localhost:9001.")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    print("The file might not be valid JSON.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Error accessing the MinIO server or file: HTTPConnectionPool(host='localhost', port=9001): Max retries exceeded with url: /browser/spotify-sample/mpd.slice.0-999.json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f96bcf16810>: Failed to establish a new connection: [Errno 111] Connection refused'))
Ensure MinIO is running and accessible at http://localhost:9001.


In [2]:
!pip install minio


In [8]:
from minio import Minio
import json

# Khởi tạo client
client = Minio("host.docker.internal:9000", access_key="minioadmin", secret_key="minioadmin", secure=False)

# Tải file JSON từ bucket
data = client.get_object("spotify-sample", "mpd.slice.0-999.json")

# Đọc nội dung JSON
playlist_data = json.load(data)
print(len(playlist_data["playlists"]))


ModuleNotFoundError: No module named 'minio'

In [11]:
playlists = playlist_data['playlists']
records = []

for p in playlists:
    for t in p['tracks']:
        records.append({
            'playlist_id': p['pid'],
            'playlist_name': p.get('name', ''),
            'track_name': t.get('track_name', ''),
            'artist_name': t.get('artist_name', ''),
            'album_name': t.get('album_name', '')
        })

df = pd.DataFrame(records)

print("✅ Số lượng playlist:", df['playlist_id'].nunique())
print("✅ Tổng số track entries:", len(df))
print("✅ Số lượng track duy nhất:", df['track_name'].nunique())
print(df.head())

✅ Số lượng playlist: 1000
✅ Tổng số track entries: 67503
✅ Số lượng track duy nhất: 30049
   playlist_id playlist_name                                  track_name  \
0            0    Throwbacks  Lose Control (feat. Ciara & Fat Man Scoop)   
1            0    Throwbacks                                       Toxic   
2            0    Throwbacks                               Crazy In Love   
3            0    Throwbacks                              Rock Your Body   
4            0    Throwbacks                                It Wasn't Me   

         artist_name                                    album_name  
0      Missy Elliott                                  The Cookbook  
1     Britney Spears                                   In The Zone  
2            Beyoncé  Dangerously In Love (Alben für die Ewigkeit)  
3  Justin Timberlake                                     Justified  
4             Shaggy                                      Hot Shot  


In [12]:
avg_tracks = df.groupby('playlist_id')['track_name'].count().mean()
print(f"Số bài hát trung bình mỗi playlist: {avg_tracks:.2f}")

# Nghệ sĩ phổ biến nhất
top_artists = df['artist_name'].value_counts().head(10)
print("\nTop 10 nghệ sĩ phổ biến nhất:")
print(top_artists)

# Album phổ biến nhất
top_albums = df['album_name'].value_counts().head(10)
print("\nTop 10 album phổ biến nhất:")
print(top_albums)

Số bài hát trung bình mỗi playlist: 67.50

Top 10 nghệ sĩ phổ biến nhất:
artist_name
Drake             939
Kanye West        415
Kendrick Lamar    385
Rihanna           350
Eminem            332
The Weeknd        296
Lil Uzi Vert      292
Ed Sheeran        285
Future            265
Chris Brown       259
Name: count, dtype: int64

Top 10 album phổ biến nhất:
album_name
Views                        234
Stoney                       184
DAMN.                        152
Greatest Hits                149
Beauty Behind The Madness    146
Original Album Classics      138
Blurryface                   134
More Life                    127
American Teen                122
The Life Of Pablo            121
Name: count, dtype: int64


In [4]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Cấu hình chuẩn để chạy với MinIO trong Docker
spark = SparkSession.builder \
    .appName("Spotify Production Pipeline") \
    .config("spark.jars", "/home/jovyan/jars/hadoop-aws-3.3.4.jar,/home/jovyan/jars/aws-java-sdk-bundle-1.12.262.jar") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.caseSensitive", "true") \
    .getOrCreate()

print("Spark Session Ready!")

Spark Session Ready!


In [6]:
# 1. Định nghĩa Schema chuẩn của Spotify MPD
# Việc này giúp Spark đọc nhanh hơn và loại bỏ bản ghi sai format ngay từ đầu
track_schema = StructType([
    StructField("pos", IntegerType(), True),
    StructField("artist_name", StringType(), True),
    StructField("track_uri", StringType(), True),
    StructField("artist_uri", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("album_uri", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("album_name", StringType(), True)
])

playlist_schema = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", LongType(), True),
    StructField("modified_at", LongType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", ArrayType(track_schema), True), # Nested Array
    StructField("description", StringType(), True)
])

root_schema = StructType([
    StructField("info", MapType(StringType(), StringType()), True),
    StructField("playlists", ArrayType(playlist_schema), True)
])

print("Loading Raw Data from Data Lake...")
# Đọc dữ liệu với schema đã định nghĩa
raw_df = spark.read \
    .schema(root_schema) \
    .option("mode", "PERMISSIVE") \
    .option("multiline", "true") \
    .json("s3a://spotify-raw-data/mpd.slice.0-999.json") # Demo 1 file

print(f"Raw Data Loaded.")

NameError: name 'StructType' is not defined

In [6]:
print("Processing Silver Layer...")

# 1. Explode Playlists (Tách mảng playlist thành từng dòng)
playlists_df = raw_df.select(explode(col("playlists")).alias("p"))

# 2. Explode Tracks (Tách mảng tracks lồng bên trong playlist)
# Đây là bước "Explode nested track arrays" trong yêu cầu
flat_df = playlists_df.select(
    col("p.pid").alias("playlist_id"),
    col("p.name").alias("playlist_name"),
    col("p.num_tracks"),
    col("p.num_followers"),
    col("p.modified_at"),
    explode(col("p.tracks")).alias("t")
)

# 3. Parse & Normalize Metadata (Làm sạch)
# - Trim: Cắt khoảng trắng thừa
# - Lower: Chuyển về chữ thường để chuẩn hóa
# - Cast: Đảm bảo đúng kiểu dữ liệu
silver_df = flat_df.select(
    col("playlist_id"),
    trim(col("playlist_name")).alias("playlist_name"),
    col("num_tracks"),
    col("num_followers"),
    # Convert timestamp (modified_at là epoch seconds)
    to_timestamp(col("modified_at")).alias("modified_date"),
    
    # Track Metadata Parsing
    trim(col("t.track_uri")).alias("track_uri"),
    trim(col("t.track_name")).alias("track_name"),
    trim(col("t.artist_uri")).alias("artist_uri"),
    trim(col("t.artist_name")).alias("artist_name"),
    col("t.duration_ms"),
    
    # Thêm cột partition key (Ví dụ: lấy chữ cái đầu của artist để partition cho đều)
    substring(col("t.artist_name"), 0, 1).alias("artist_partition")
).drop_duplicates() # Loại bỏ dòng trùng lặp (Deduplication)

# Lưu bảng sạch này xuống Data Lake (Silver)
silver_df.write.mode("overwrite").parquet("s3a://warehouse/silver/tracks")
print("Silver Layer (Cleaned & Normalized) Saved.")
silver_df.show(3)

🔄 Processing Silver Layer...
✅ Silver Layer (Cleaned & Normalized) Saved.
+-----------+----------------+----------+-------------+-------------------+--------------------+----------------+--------------------+-------------------+-----------+----------------+
|playlist_id|   playlist_name|num_tracks|num_followers|      modified_date|           track_uri|      track_name|          artist_uri|        artist_name|duration_ms|artist_partition|
+-----------+----------------+----------+-------------+-------------------+--------------------+----------------+--------------------+-------------------+-----------+----------------+
|          0|      Throwbacks|        52|            1|2017-04-29 00:00:00|spotify:track:2eJ...|            Baby|spotify:artist:1u...|      Justin Bieber|     213973|               J|
|          1|Awesome Playlist|        39|            1|2017-09-28 00:00:00|spotify:track:2HH...|Eye of the Tiger|spotify:artist:26...|           Survivor|     243773|               S|
|     

In [7]:
print("Processing Gold Layer (Feature Engineering)...")

# Feature 1: Artist Frequency (Độ phổ biến của nghệ sĩ)
# Nghệ sĩ này xuất hiện trong bao nhiêu playlist? Tổng thời lượng nghe là bao nhiêu?
artist_features = silver_df.groupBy("artist_name") \
    .agg(
        countDistinct("playlist_id").alias("total_playlist_appearances"),
        count("track_uri").alias("total_tracks_count"),
        avg("duration_ms").alias("avg_track_duration"),
        sum("duration_ms").alias("total_listen_time_ms")
    ) \
    .withColumn("popularity_score", col("total_playlist_appearances") * 0.7 + col("total_tracks_count") * 0.3)

# Feature 2: Playlist Complexity (Độ phức tạp của Playlist)
# Playlist này có bao nhiêu nghệ sĩ khác nhau? Độ đa dạng thế nào?
playlist_features = silver_df.groupBy("playlist_id", "playlist_name") \
    .agg(
        count("track_uri").alias("playlist_length"),
        countDistinct("artist_name").alias("unique_artists"),
        avg("duration_ms").alias("avg_track_duration")
    ) \
    .withColumn("diversity_ratio", col("unique_artists") / col("playlist_length"))

print("Sample Artist Features:")
artist_features.orderBy(desc("total_playlist_appearances")).show(5)

print("Sample Playlist Features:")
playlist_features.orderBy(desc("diversity_ratio")).show(5)

Processing Gold Layer (Feature Engineering)...
Sample Artist Features:
+----------------+--------------------------+------------------+------------------+--------------------+------------------+
|     artist_name|total_playlist_appearances|total_tracks_count|avg_track_duration|total_listen_time_ms|  popularity_score|
+----------------+--------------------------+------------------+------------------+--------------------+------------------+
|           Drake|                       202|               923|237832.82665222103|           219519699|418.29999999999995|
|         Rihanna|                       170|               348| 224016.5459770115|            77957758|223.39999999999998|
|      Kanye West|                       149|               412|248709.39077669903|           102468269|227.89999999999998|
|      The Weeknd|                       139|               291| 264382.6632302406|            76935355|             184.6|
|The Chainsmokers|                       121|               2

In [8]:
print("Saving Final Data (Storage Optimization)...")

# 1. Lưu Artist Features (Partition theo chữ cái đầu để truy vấn nhanh)
# Lưu ý: Partition by artist_id/name trực tiếp sẽ tạo ra hàng triệu folder nhỏ (Bad Practice).
# Chuẩn chỉ là nên partition theo nhóm (Bucketing) hoặc chữ cái đầu.
artist_features.write \
    .mode("overwrite") \
    .partitionBy("total_playlist_appearances") \
    .parquet("s3a://warehouse/gold/artist_features")

# 2. Lưu Playlist Features (Partition by Range ví dụ theo độ dài playlist)
# Tạo bucket cột playlist_length để partition cho gọn
playlist_features_bucketed = playlist_features.withColumn(
    "length_bucket", 
    (col("playlist_length") / 50).cast("integer") * 50 # Gom nhóm 0-50, 50-100...
)

playlist_features_bucketed.write \
    .mode("overwrite") \
    .partitionBy("length_bucket") \
    .parquet("s3a://warehouse/gold/playlist_features")

print("All Data Saved & Partitioned Successfully!")

Saving Final Data (Storage Optimization)...
All Data Saved & Partitioned Successfully!


In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col

# --- 1. KHỞI TẠO SPARK (Cấu hình chuẩn cho Docker/MinIO) ---
# Không cần thay đổi gì ở đây, đây là "chìa khóa" để chạy trong Docker
JAR_PATH = "/home/jovyan/jars"
jars = [
    f"{JAR_PATH}/hadoop-aws-3.3.4.jar",
    f"{JAR_PATH}/aws-java-sdk-bundle-1.12.262.jar"
]

spark = SparkSession.builder \
    .appName("Spotify ETL Exact Logic") \
    .config("spark.jars", ",".join(jars)) \
    .config("spark.driver.memory", "4g") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
print("✅ Spark Session Ready!")


✅ Spark Session Ready!


In [3]:
import os
import time
# Lưu ý: Không cần import glob nữa vì ta dùng Hadoop FS

# --- 2. LẤY DANH SÁCH FILE TỪ MINIO (CÁCH FIX LỖI WRONG FS) ---
print("🔎 Scanning MinIO files...")

# Lấy các class Java cần thiết qua Py4J
Path = spark._jvm.org.apache.hadoop.fs.Path
URI = spark._jvm.java.net.URI
FileSystem = spark._jvm.org.apache.hadoop.fs.FileSystem
Configuration = spark._jsc.hadoopConfiguration()

# 1. Định nghĩa đường dẫn S3
bucket_path_str = "s3a://spotify-raw-data/"
p = Path(bucket_path_str)

# 2. Lấy FileSystem TƯƠNG ỨNG với đường dẫn đó (Đây là bước sửa lỗi quan trọng)
# Nó sẽ tự động nhận diện giao thức s3a:// và trả về S3AFileSystem thay vì LocalFileSystem
fs = p.getFileSystem(Configuration)

# 3. Liệt kê file
file_statuses = fs.listStatus(p)
s3_files = [f.getPath().toString() for f in file_statuses if f.getPath().getName().endswith('.json')]
s3_files.sort()

print(f"📂 Tìm thấy {len(s3_files)} files. Bắt đầu xử lý...")

# --- 3. VÒNG LẶP XỬ LÝ (LOGIC CỦA BẠN) ---
output_path = "s3a://spotify-processed-data/"
count = 0

for file_path in s3_files:
    count += 1
    # Lấy tên file để in log cho đẹp
    file_name = file_path.split("/")[-1]
    
    print(f"🔄 [{count}/{len(s3_files)}] Processing: {file_name} ...", end=" ")
    
    try:
        # 1. Đọc Data (Read)
        # Lưu ý: file_path lấy từ Hadoop FS đã là chuẩn s3a://... rồi, dùng luôn
        df = spark.read.option("multiline", "true").json(file_path)
        
        # Kiểm tra xem file có dữ liệu không
        if "playlists" in df.columns:
            
            # --- [LOGIC BIẾN ĐỔI CỦA BẠN] ---
            df_playlists = df.select(explode("playlists").alias("playlist"))

            df_tracks = df_playlists.select(
                col("playlist.pid").alias("pid"),
                col("playlist.name").alias("playlist_name"),
                explode("playlist.tracks").alias("track")
            )

            final_df = df_tracks.select(
                "pid",
                "playlist_name",
                col("track.track_name").alias("track_name"),
                col("track.track_uri").alias("track_uri"),
                col("track.artist_name").alias("artist_name"),
                col("track.album_name").alias("album_name"),
                col("track.duration_ms").alias("duration_ms")    
            )
            
            # --- [GHI DỮ LIỆU] ---
            # File đầu tiên thì Overwrite (xóa cũ), các file sau Append (nối thêm)
            mode = "overwrite" if count == 1 else "append"
            final_df.write.mode(mode).parquet(output_path)
            
            # Dọn RAM
            df.unpersist()
            print("✅ OK")
            
        else:
            print("⚠️ Skipped (No 'playlists' key)")

    except Exception as e:
        print(f"❌ Error: {e}")

print("\n🎉 ETL JOB COMPLETED SUCCESSFULLY!")

# --- 4. KIỂM TRA KẾT QUẢ ---
print("🔍 Kết quả mẫu trong MinIO:")
try:
    spark.read.parquet(output_path).show(5, truncate=False)
except:
    print("Chưa có dữ liệu để hiển thị.")

🔎 Scanning MinIO files...
📂 Tìm thấy 100 files. Bắt đầu xử lý...
🔄 [1/100] Processing: mpd.slice.114000-114999.json ... ❌ Error: Can not create a Path from an empty string
🔄 [2/100] Processing: mpd.slice.115000-115999.json ... ✅ OK
🔄 [3/100] Processing: mpd.slice.117000-117999.json ... ✅ OK
🔄 [4/100] Processing: mpd.slice.124000-124999.json ... ✅ OK
🔄 [5/100] Processing: mpd.slice.126000-126999.json ... ✅ OK
🔄 [6/100] Processing: mpd.slice.127000-127999.json ... ✅ OK
🔄 [7/100] Processing: mpd.slice.191000-191999.json ... ✅ OK
🔄 [8/100] Processing: mpd.slice.192000-192999.json ... ✅ OK
🔄 [9/100] Processing: mpd.slice.193000-193999.json ... ✅ OK
🔄 [10/100] Processing: mpd.slice.214000-214999.json ... ✅ OK
🔄 [11/100] Processing: mpd.slice.216000-216999.json ... ✅ OK
🔄 [12/100] Processing: mpd.slice.217000-217999.json ... ✅ OK
🔄 [13/100] Processing: mpd.slice.22000-22999.json ... ✅ OK
🔄 [14/100] Processing: mpd.slice.224000-224999.json ... ✅ OK
🔄 [15/100] Processing: mpd.slice.225000-22599

In [4]:
import sys
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline


# --- 2. LOGIC TRAINING ---
def run_model_training_job():
    # Sử dụng dữ liệu đã xử lý từ bước trước
    # Lưu ý: Đảm bảo bạn đã chạy bước ETL và có dữ liệu tại đây
    input_path = "s3a://spotify-processed-data/"
    
    # Lưu Model vào bucket warehouse cho chắc ăn (tránh lỗi thiếu bucket)
    model_output_path = "s3a://warehouse/models/als_model"

    print(f"⏳ Đang đọc dữ liệu từ: {input_path}")
    
    try:
        df = spark.read.parquet(input_path)
        
        # In schema để kiểm tra
        df.printSchema()

        # Feature Engineer: Tạo cột rating giả định (Implicit Feedback)
        # Vì dữ liệu chỉ có nghe/không nghe, ta gán rating = 1.0 cho mọi tương tác
        print("🔄 Đang xử lý Features...")
        
        # Chuyển đổi String ID sang Numeric Index (Yêu cầu bắt buộc của ALS)
        # setHandleInvalid("skip") giúp tránh lỗi nếu gặp ID lạ
        pid_indexer = StringIndexer(inputCol="pid", outputCol="pid_numeric", handleInvalid="skip")
        track_indexer = StringIndexer(inputCol="track_uri", outputCol="track_numeric", handleInvalid="skip")
        
        df_with_rating = df.withColumn("rating", lit(1.0))

        # Split Data
        print("✂️ Đang chia tập Train/Test...")
        (training_data, validation_data) = df_with_rating.randomSplit([0.8, 0.2], seed=42)
        
        # Cache để chạy nhanh hơn
        training_data.cache()
        validation_data.cache()

        # Định nghĩa Model ALS
        print("🤖 Đang cấu hình Model ALS...")
        als = ALS(
            userCol="pid_numeric",
            itemCol="track_numeric",
            ratingCol="rating",
            coldStartStrategy="drop", # Quan trọng: Bỏ qua user/item mới không có trong train
            implicitPrefs=True,
            rank=10,
            maxIter=5, # Demo để 5 cho nhanh (Thực tế có thể để 10)
            regParam=0.1
        )

        # Tạo Pipeline
        # LƯU Ý: Đã sửa lỗi chính tả 'all' thành 'als' ở đây
        pipeline = Pipeline(stages=[pid_indexer, track_indexer, als])

        # Train Model
        print("🏋️‍♂️ Đang Train Model (Có thể mất vài phút)...")
        model = pipeline.fit(training_data)
        print("✅ Train xong!")

        # Evaluate
        print("📉 Đang đánh giá Model...")
        predictions = model.transform(validation_data)
        
        evaluator = RegressionEvaluator(
            metricName="rmse",
            labelCol="rating",
            predictionCol="prediction"
        )
        rmse = evaluator.evaluate(predictions)
        print(f"🎯 Root Mean Squared Error (RMSE) = {rmse}")

        # Save Model
        print(f"💾 Đang lưu Model xuống: {model_output_path}")
        model.write().overwrite().save(model_output_path)

        print("🎉 Model training job completed successfully!")
        
        # Unpersist để giải phóng RAM
        training_data.unpersist()
        validation_data.unpersist()

    except Exception as e:
        print(f"❌ Lỗi: {e}")
        import traceback
        traceback.print_exc()

# --- 3. CHẠY JOB ---
run_model_training_job()

⏳ Đang đọc dữ liệu từ: s3a://spotify-processed-data/
root
 |-- pid: long (nullable = true)
 |-- playlist_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- track_uri: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- album_name: string (nullable = true)
 |-- duration_ms: long (nullable = true)

🔄 Đang xử lý Features...
✂️ Đang chia tập Train/Test...
🤖 Đang cấu hình Model ALS...
🏋️‍♂️ Đang Train Model (Có thể mất vài phút)...
✅ Train xong!
📉 Đang đánh giá Model...
🎯 Root Mean Squared Error (RMSE) = 0.945421250432738
💾 Đang lưu Model xuống: s3a://warehouse/models/als_model
🎉 Model training job completed successfully!


In [2]:
!pip install redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.2/354.2 kB 1.6 MB/s eta 0:00:00a 0:00:01m


In [3]:
import os
import redis
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.sql.functions import col, collect_list, explode
# --- 2. HÀM GHI VÀO REDIS ---
# (Thay thế cho spark_jobs.utils.write_to_redis)
# Hàm ghi Redis (Giữ nguyên)
def write_to_redis(iterator):
    r = redis.Redis(host='redis', port=6379, db=0)
    pipe = r.pipeline()
    count = 0
    for row in iterator:
        key = f"playlist:{row['pid']}"
        value = ",".join(row['recommendations'])
        pipe.set(key, value)
        count += 1
        if count % 100 == 0: # Giảm batch size xuống cho nhẹ
            pipe.execute()
            count = 0
    if count > 0:
        pipe.execute()
    r.close()

# --- 2. LOGIC CHÍNH ---
def run_safe_job():
    model_path = "s3a://warehouse/models/als_model"
    
    print(f"⏳ Loading Model...")
    try:
        model = PipelineModel.load(model_path)
    except Exception as e:
        print("❌ Chưa có model. Hãy chạy bước Train trước!")
        return

    als_model = model.stages[-1]
    user_col_name = als_model.getUserCol() # pid_numeric

    # === [QUAN TRỌNG] CHỈ LẤY MẪU USER ===
    print("✂️ Lấy mẫu 1% danh sách Playlist để tính toán...")
    
    # Lấy danh sách ID số từ metadata mapping đã lưu trong model
    pid_indexer = model.stages[0]
    # Tạo DataFrame chứa tất cả User ID số
    all_users = spark.range(0, len(pid_indexer.labels)).withColumnRenamed("id", user_col_name)
    
    # Chỉ lấy 1% (hoặc tối đa 100 user) để demo
    subset_users = all_users.sample(fraction=0.01, seed=42).limit(100)
    
    print(f"🤖 Đang tạo gợi ý cho {subset_users.count()} playlist mẫu...")
    
    # Chỉ tính toán cho tập con này (Nhẹ hơn gấp trăm lần so với recommendForAllUsers)
    recommendations = als_model.recommendForUserSubset(subset_users, 10)

    # --- ÁNH XẠ (MAPPING) ---
    print("🔄 Mapping IDs...")
    
    pid_labels = pid_indexer.labels
    track_labels = model.stages[1].labels

    # Tạo Map DF
    pid_map_df = spark.createDataFrame(list(enumerate(pid_labels)), ["pid_numeric", "pid"])
    track_map_df = spark.createDataFrame(list(enumerate(track_labels)), ["track_numeric", "track_uri"])

    # Join lấy PID
    recs_with_pid = recommendations.join(pid_map_df, user_col_name, "inner") \
        .select("pid", "recommendations")

    # Explode & Join lấy Track URI
    exploded = recs_with_pid.select("pid", explode("recommendations").alias("rec")) \
        .select("pid", col("rec." + als_model.getItemCol()).alias("track_numeric"))

    joined_tracks = exploded.join(track_map_df, "track_numeric", "inner") \
        .select("pid", "track_uri")

    # Gom lại
    final_recs = joined_tracks.groupBy("pid") \
        .agg(collect_list("track_uri").alias("recommendations"))

    print("👀 Kết quả mẫu:")
    final_recs.show(3, truncate=False)

    # --- GHI REDIS ---
    print("🚀 Đẩy vào Redis...")
    final_recs.foreachPartition(write_to_redis)
    print("✅ Xong! Kiểm tra Redis.")
    
    # Test
    r = redis.Redis(host='redis', port=6379, db=0)
    if final_recs.count() > 0:
        pid = final_recs.first()['pid']
        print(f"🔍 Redis Check Key [playlist:{pid}]: {r.get(f'playlist:{pid}')}")

run_safe_job()

⏳ Loading Model...
✂️ Lấy mẫu 1% danh sách Playlist để tính toán...
🤖 Đang tạo gợi ý cho 100 playlist mẫu...
🔄 Mapping IDs...
👀 Kết quả mẫu:
+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|pid   |recommendations                                                                                                                                                                                                                                                                                                                                                                             |
+------+-----------------------------------------------------------------------

In [11]:
spark.stop()

In [8]:
r = redis.Redis(host='redis', port=6379, db=0)
print(f"🔍 Redis Check Key [playlist:]: {r.get(f'playlist:{117816}')}")

🔍 Redis Check Key [playlist:]: b'spotify:track:7yq4Qj7cqayVTp3FF9CWbm,spotify:track:3rq5w4bQGigXOfdN30ATJt,spotify:track:1jdNcAD8Ir58RlsdGjJJdx,spotify:track:2ihCaVdNZmnHZWt0fvAM7B,spotify:track:0GO8y8jQk1PkHzS31d699N,spotify:track:6vECYJHxYmm3Ydt3fF01pE,spotify:track:6ZYS6QQxTLsQ6IFXdVx1r4,spotify:track:3n69hLUdIsSa1WlRmjMZlW,spotify:track:2QjOHCTQ1Jl3zawyYOpxh6,spotify:track:7w5cxTEzp1rfV3KCy0Bd5N'


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 38332)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =